# Assignment 1

# 4. Exact Diagonalization study of the quantum Ising model

# 4.1 Dense ED

Generate the quantum Ising Hamiltonian (4) as a dense matrix and call an explicit diagonalization routine for the entire spectrum for system sizes $L = 8, 10, 12, 14,$ and for a range of values of h. Plot the ground state energy as a function of $h$ for the various $L$. Compare the open systems with periodic ones for the same parameters—how does each phase react to the boundaries?


import relevant packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from tqdm import tqdm
import time
import sys
import os
from multiprocessing import Pool
from functools import partial
directory = 'figures'
if not os.path.exists(directory):
    os.makedirs(directory)
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm.notebook import tqdm
import numpy as np
from dask.distributed import Client, progress
from dask import compute, delayed
import dask.array as da
from dask.diagnostics import ProgressBar
plt.rcParams['figure.dpi']=400

## Constructing the  dense Hamiltonian

The Hamiltonian of the quantum Ising chain is given by

$$H = -J \sum_{j=1}^{L-1} \sigma_z^j \sigma_z^{j+1} - h \sum_{j=1}^{L} \sigma_x^j $$

Instead of having to do all these time-consuming Kronecker products we can quickly realize that 

$$		H_{\alpha \beta} = \langle e_\alpha | H | e_\beta \rangle \neq 0 \quad \text{if} \quad 
		\begin{cases}
			\alpha = \beta \\
			\text{or } \alpha \text{ and } \beta \text{ differ by a single bit flip.}
		\end{cases}$$
        
This is because $|\uparrow \rangle$ and $|\downarrow \rangle$ are eigenstates of $\sigma_z$ and $\sigma_x$ flips $|\uparrow \rangle \longleftrightarrow |\downarrow \rangle$. The states $| e_\alpha \rangle$  are defined in an orthogonal basis as follows:


\begin{align*}
|e_0\rangle &= |000 \ldots 0\rangle, \\
|e_1\rangle &= |100 \ldots 0\rangle, \\
|e_2\rangle &= |010 \ldots 0\rangle, \\
|e_3\rangle &= |110 \ldots 0\rangle, \\
&\text{and so on.}
\end{align*}


Where $0$ represents $|\uparrow \rangle$ and $1$ represents $|\downarrow \rangle$. Now that we have the states represented as integers, we can use fast bit operations to quickly construct the Hamiltonian.

In [ ]:
def denseH(L, J, h, periodic):
    """
    generates the dense Hamiltonian matrix for the quantum Ising chain
    
        Parameters:
            L (int): length of chain
            J (float): ising interaction strength
            h (float): magnetic field strength
            periodic (bool): does the chain have periodic boundary conditions?
            
        Returns:
            H (ndarray): 2^L x 2^L matrix representing the Hamiltonian operator
    """

    dim=2 ** L # dimensions of the Hilbert space
    
    H=np.zeros((dim, dim)) # initliaze the Hamiltonian
    
    "Calculation of off-diagonal elements due to the magnetic field"
    
    for beta in range(dim): # iterate over all states
        
        for j in range(1,L+1): # iterate over all sites
            
            alpha = beta ^ (1<<j-1) # flips jth bit of beta to get the state alpha that is related to beta by a single bit flip
            
            H[alpha, beta] -= h # contribution by sigma^j_x
            
    "Calculation of diagonal elements due to Ising interaction"

    for alpha in range(dim): # iterate over all states
        
        for j in range(1, L): # iterate over all sites
            
            if 2*(alpha & (1 << j-1)) == alpha & (1 << j): # check if site j and j+1 have the same spin
                
                H[alpha, alpha] -= J # if they do, decrease the energy by the ising interaction term
                
            else:
                
                H[alpha, alpha] += J # if not, increase the energy by the ising interaction term
        
        "Handling case of periodic boundary conditions"
                
        if periodic and L > 1: # L > 1 needed for periodicity to mean anything
            
            if (alpha & (1 << L-1)) == ((alpha & (1 << 0))*(2**(L-1))): # Check if the states at either end have the same spin
                
                H[alpha, alpha] -= J # if they do, decrease the energy by the ising interaction term
                
            else:
                
                H[alpha, alpha] += J # if not, increase the energy by the ising interaction term
                
    return H   

### Dense diagonalization implementation

In [ ]:
def denseEgs(L, J, h, periodic):
    """
    returns the ground state eigenenergy of the Ising chain
    
        Parameters:
            L (int): length of chain
            J (float): ising interaction strength
            h (float): magnetic field strength
            periodic (bool): does the chain have periodic boundary conditions?
        
        Returns:
            ground_state (float): ground state energy
    """

    H = denseH(L, J, h , periodic) # construct the dense Hamiltonian
    
    ground_state = scipy.linalg.eigh(H, subset_by_index=(0, 0), eigvals_only=True)[0] # return only the smallest eigenvalue (increases the speed quite a bit)
    
    return ground_state 

### Plotting the ground state energy as a function of h for various L

In [ ]:
# initialize
L_values = [8, 10, 12, 14] 
h_values = np.linspace(0, 2, 20) # the problem is symmetric about h = 0, so we can cut computation-time in half by computing only positive h
periodicgs = {L: [] for L in L_values}
opengs = {L: [] for L in L_values}
xkcd_colors=['xkcd:indigo', 'xkcd:royal blue', 'xkcd:bright green', 'xkcd:red']

"Plot Egs vs h for both periodic and open boundary conditions"

for L in L_values:
    for h in tqdm(h_values, desc=f'Calculating for L={L}'):
        periodicgs[L].append(denseEgs(L, 1, h, True))
        opengs[L].append(denseEgs(L, 1, h, False))
        
# Dashed line -> Open
# Solid line -> Periodic
        
for i, (L, energies) in enumerate(periodicgs.items()):
    plt.plot(h_values, energies, label=f'L={L}', color=xkcd_colors[i])
    
for i, (L, energies) in enumerate(opengs.items()):
    plt.plot(h_values, energies, color=xkcd_colors[i], linestyle='--')

plt.xlabel('field strength')
plt.ylabel(r'$E_{gs}$')
plt.title('Ground state energy versus magnetic field \n for various values of L')
plt.legend()
plt.savefig(os.path.join(directory, 'plot-dense.png'), dpi=400)
plt.show()

# 4.2 Sparse ED

Construct the same Hamiltonian as a sparse matrix and use a sparse matrix diagonalization routine
provided by a standard library. Obtain the ground state and a few excited states, and for small
system sizes verify your results against the dense ED solution. How large of a system can you push
the sparse diagonalization routine to solve?

Optional. Try implementing your own Lanczos routine.

As you can see L = 14 takes forever to run and the memory limits significantly constrain us as well. But, most of the elements of the Hamiltonian are zero, we can use this sparseness to our advantage by slightly modifying our code. Instead of initializing the whole $2^L \times 2^L$ matrix, we just keep track of the non-zero elements of the Hamiltonian and use scipy.sparse to convert it into a standard sparse-matrix data structure.

## Constructing the Sparse Hamiltonian

In [ ]:
def sparseH(L, J, h, periodic):
    
    """
    generates the sparse Hamiltonian matrix for the quantum Ising chain
    
        Parameters:
            L (int): length of chain
            J (float): ising interaction strength
            h (float): magnetic field strength
            periodic (bool): does the chain have periodic boundary conditions?
            
        Returns:
            H (csr_matrix): sparse matrix representing the Hamiltonian operator
    """
    
    dim = 2 ** L # dimensions of the Hilbert space
    
    # initialize 
    H_data = []
    H_rows = []
    H_cols = []
    
    "Calculation of off-diagonal elements due to the magnetic field"
    
    for beta in range(dim): # iterate over all states
        
        for j in range(1, L + 1): # iterate over all sites
            
            alpha = beta ^ (1 << (j - 1)) # flips jth bit of beta to get the state alpha that is related to beta by a single bit flip
            
            "Keep track of the indices with non-zero matrix elements"
            
            H_data.append(-h)
            H_rows.append(alpha)
            H_cols.append(beta)
    
    "Calculation of diagonal elements due to Ising interaction"

    for alpha in range(dim):  # iterate over all states
        
        A = 0
        
        for j in range(1, L): # iterate over all sites
            
            if 2 * (alpha & (1 << (j - 1))) == alpha & (1 << j): # check if site j and j+1 have the same spin
                
                A -= J # if they do, decrease the energy by the ising interaction term
                
            else:
                
                A += J # if not, increase the energy by the ising interaction term
                
        "Handling periodic boundary conditions"
                
        if periodic and L > 1: # L > 1 needed for periodicity to mean anything
            
            if (alpha & (1 << (L - 1))) == ((alpha & (1 << 0)) * (2 ** (L - 1))): # Check if the states at either end have the same spin
                
                A -= J # if they do, decrease the energy by the ising interaction term
                
            else:
                
                A += J # if not, increase the energy by the ising interaction term
        
        if A != 0: # Check if the resulting matrix element is non-zero, if so, keep track of it
        
            H_data.append(A)
            H_rows.append(alpha)
            H_cols.append(alpha)

    H_data = np.array(H_data, dtype=float) # convert the list into a np array
    
    H = scipy.sparse.csr_matrix((H_data, (H_rows, H_cols)), shape=(dim, dim), dtype=np.float64) # make it into a csr sparse matrix
    
    return H

## Consistency between Sparse and Dense Diagonalizers

Let me write up some useful functions

In [ ]:
def diagonalize_dense(L, J, h, periodic):
    """
    generates the eigenstates and eigenenergies from the dense ED
    
        Parameters:
            L (int): length of chain
            J (float): ising interaction strength
            h (float): magnetic field strength
            periodic (bool): does the chain have periodic boundary conditions?
            
        Returns:
            energies: eigenenergies
            states: eigenvectors
    """

    energies, states = scipy.linalg.eigh(denseH(L, J, h, periodic))
    return energies, states

def diagonalize_sparse(L, J, h, periodic, num_states=6):
    """
    generates the eigenstates and eigenenergies from the dense ED
    
        Parameters:
            L (int): length of chain
            J (float): ising interaction strength
            h (float): magnetic field strength
            periodic (bool): does the chain have periodic boundary conditions?
            num_states = number of states needed
            
        Returns:
            energies: eigenenergies
            states: eigenvectors
    """
    energies, states = scipy.sparse.linalg.eigsh(sparseH(L, J, h, periodic), k=num_states, which='SA')
    return energies, states

def compare_states(states_dense, states_sparse):
    """
    computes the overlap between corresponding eigenstates obtained from dense and sparse diagonalization methods.
    
    Parameters:
        states_dense (np.ndarray): a 2D array containing the eigenvectors obtained from the dense diagonalization.

        states_sparse (np.ndarray): a 2D array containing the eigenvectors obtained from the sparse diagonalization.
                                    
    Returns:
        overlaps (list): list of overlap values for corresponding eigenstates. Each overlap value is a float between
                         0 and 1.
    """
    
    overlaps = []
    
    for i in range(len(states_sparse[0])):
        overlap = abs(np.dot(states_dense[:, i].conjugate(), states_sparse[:, i])) 
        # calculates the absolute overlap between the i-th eigenvectors from the dense and sparse results
        
        overlaps.append(overlap)
        
    return overlaps

In [ ]:
L_values = [8, 10]
h_values = np.linspace(0, 1, 5)
num_states = 6

for L in L_values:
    for h in h_values:
        
        energies_dense, states_dense = diagonalize_dense(L, 1, h, True)
        energies_sparse, states_sparse = diagonalize_sparse(L, 1, h, True, num_states=num_states)
        
        print(f"L={L}, h={h}")
        print("Dense ED Energies:", energies_dense[:num_states])
        print("Sparse ED Energies:", energies_sparse)
        
        overlaps = compare_states(states_dense, states_sparse)
        print("State overlaps:", overlaps)


As you can see the eigenenergies do match, but the eigenvectors do not seem to have perfect overlap, this is because the states are degenerate and any linear combination can be reported as an eigenvector. You can see this clearly as the non-degenerate states do seem to match.

## Implementation of the sparse matrix as a functional pointer 

The code below might be useful if there are memory constraints, as instead of writing the Hamiltonian as a matrix we just write up rules that give $H | \psi \rangle$ given $| \psi \rangle$. This method however seems to be less time-efficient to diagonalize.

In [ ]:
def H_operator(L, J, h, periodic):
    """
    generates the Hamiltonian operator for the quantum Ising chain
    
        Parameters:
            L (int): length of chain
            J (float): ising interaction strength
            h (float): magnetic field strength
            periodic (bool): does the chain have periodic boundary conditions?
            
        Returns:
            H (LinOperator): function pointer representing the Hamiltonian operator
    """
    def H_psi(psi):
        """
        Generates Hpsi given psi
        """
        dim = 2 ** L
        Hpsi = np.zeros(dim, dtype=np.float64)
        
        
        "Calculation of off-diagonal elements due to the magnetic field"
        
        for beta in range(dim): # iterate over all states
        
            for j in range(1, L + 1): # iterate over all sites
                
                alpha = beta ^ (1 << (j - 1)) # flips jth bit of beta to get the state alpha that is related to beta by a single bit flip
                
                Hpsi[alpha] -= h * psi[beta] # change the coefficient of the alpha state
                
        
        "Calculation of diagonal elements due to Ising interaction"
        
        for alpha in range(dim): # iterate over all states
            
            A = 0

            for j in range(1, L): # iterate over all sites

                if 2 * (alpha & (1 << (j - 1))) == alpha & (1 << j):  # check if site j and j+1 have the same spin
                    
                    A -= J # if they do, decrease the energy by the ising interaction term
                    
                else:
                    A += J # if not, increase the energy by the ising interaction term

            if periodic and L > 1:  # L > 1 needed for periodicity to mean anything
                
                if (alpha & (1 << (L - 1))) == ((alpha & (1 << 0)) * (2 ** (L - 1))): # Check if the states at either end have the same spin
                    
                    A -= J # if they do, decrease the energy by the ising interaction term
                    
                else:
                    A += J # if not, increase the energy by the ising interaction term

            Hpsi[alpha] += A * psi[alpha] # change the coefficient of the alpha state
    
    dim = 2 ** L
    
    return scipy.sparse.linalg.LinearOperator((dim, dim), matvec=H_psi)


Performance difference between the two styles

In [ ]:
L = 14
J = 1
h = 0.5
periodic = True
num_states = 1


start_time_sparse = time.time()
H_sparse = sparseH(L, J, h, periodic)
energies_sparse, states_sparse = scipy.sparse.linalg.eigsh(H_sparse, k=num_states, which='SA')
end_time_sparse = time.time()

start_time_operator = time.time()
H_op = H_operator(L, J, h, periodic)
energies_op, states_op = scipy.sparse.linalg.eigsh(H_op, k=num_states, which='SA')
end_time_operator = time.time()

time_sparse = end_time_sparse - start_time_sparse
time_operator = end_time_operator - start_time_operator

print('time taken by csr Hamiltonian: ', time_sparse)

print('time taken by operator Hamiltonian: ', time_operator)



H_sparse_memory = sys.getsizeof(H_sparse.data) + sys.getsizeof(H_sparse.indices) + sys.getsizeof(H_sparse.indptr)
H_op_memory = sys.getsizeof(H_op)

print('memory used by csr Hamiltonian: ', H_sparse_memory)

print('memory used by operator Hamilonian: ', H_op_memory)


Since time is our bigger concern I will use sparseH, however this functional pointer approach might be worth considering depending on the task chosen.

In [ ]:
L_values = [8, 10, 12, 14]
h_values = np.linspace(0, 2, 21)
J = 1
periodic = True

energy_levels = {L: [] for L in L_values}

for L in L_values:
    for h in h_values:
        H = sparseH(L, J, h, periodic)
        eigenvalues = scipy.sparse.linalg.eigsh(H, k=5, which='SA', return_eigenvectors=False)
        energy_levels[L].append(eigenvalues)

In [ ]:
line_styles = ['-', '--', '-.', ':']
markers = ['o', '^', 's', '*', 'p', 'D', 'x', '+']
xkcd_colors=['xkcd:indigo', 'xkcd:green', 'xkcd:red', 'xkcd:orange']

plt.figure(figsize=(12, 8))
for idx, L in enumerate(L_values):
    energies = np.array(energy_levels[L])
    for i in range(5):
        label = f'L={L}' if i == 0 else None
        plt.plot(h_values, energies[:, i], label=label,marker=markers[idx],color=xkcd_colors[idx], markevery=3, linewidth=1.5)

plt.title('Ground and Excited State Energies vs. Magnetic Field h for Various L')
plt.xlabel('h (Magnetic Field Strength)')
plt.ylabel('Energy')
plt.legend()
plt.savefig('fewexcited.png', dpi=400)
plt.show()

# 4.3 Study of convergence with system size

For representative values of $h$ inside each phase (e.g., $h = 0.3$) in the ferromagnetic phase and $h = 1.7$ in the paramagnet), study the $L$ dependence of the ground state energy per site, $E_{\text{gs}}(L)/L$, for systems with both periodic and open boundary conditions. Comment on the approach to the thermodynamic limit $L \rightarrow \infty$ for the two types of boundaries.



In [ ]:
def sparseEgs(L, J, h, periodic):
    """
    returns the ground state eigenenergy of the Ising chain using sparse diagonalization 
    
        Parameters:
            L (int): length of chain
            J (float): ising interaction strength
            h (float): magnetic field strength
            periodic (bool): does the chain have periodic boundary conditions?
        
        Returns:
            ground_state (float): ground state energy
    """

    H = sparseH(L, J, h , periodic) # construct the dense Hamiltonian
    
    ground_state = scipy.sparse.linalg.eigsh(H, k=1, which='SA', return_eigenvectors=False)[0] # return only the smallest eigenvalue (increases the speed quite a bit)
    
    return ground_state 

Let us check the convergence to thermodynamic limit of the ferromagnetic and paramagnetic phases for increasing L

In [ ]:
paraPERIODIC=[]
ferroPERIODIC=[]
paraOPEN=[]
ferroOPEN=[]
Ls = list(range(2, 18))  

for L in tqdm(Ls, desc="Calculating energies"):
    start_time = time.time()
    ferroPERIODIC.append(sparseEgs(L, 1, 0.3, True)/L)
    paraPERIODIC.append(sparseEgs(L, 1, 1.7, True)/L)
    ferroOPEN.append(sparseEgs(L, 1, 0.3, False)/L)
    paraOPEN.append(sparseEgs(L, 1, 1.7, False)/L)
    elapsed_time = time.time() - start_time
    print(f"L = {L}, Time taken: {elapsed_time:.2f} seconds")

In [ ]:
plt.plot(Ls, ferroOPEN, label='open', linestyle="dashdot", color='xkcd:royal blue')
plt.plot(Ls, ferroPERIODIC, label="periodic", color="xkcd:bright red")
plt.legend()
plt.xlabel('L')
plt.ylabel(r'$\epsilon_{gs}(L)/L$')
plt.title( r'$\epsilon_{gs}(L)/L$ dependence on $L$ for ferro-magnetic phase ($h=0.3$)')
plt.savefig(os.path.join(directory, 'ferro_open_periodic.png'), dpi=400)
plt.show()



plt.plot(Ls, paraOPEN, label='open', linestyle="dashdot", color='xkcd:royal blue')
plt.plot(Ls, paraPERIODIC, label="periodic", color="xkcd:bright red")
plt.title( r'$\epsilon_{gs}(L)/L$ dependence on $L$ for para-magnetic phase ($h=1.7$)')
plt.xlabel('L')
plt.ylabel(r'$\epsilon_{gs}(L)/L$')
plt.legend()
plt.savefig(os.path.join(directory, 'para_open_periodic.png'), dpi=400)
plt.show()

### Plotting $\frac{E(L+2)-E(L)}{2}$



In [ ]:
client = Client(n_workers=20)
ProgressBar().register()

In [ ]:
@delayed
def delayed_sparseEgs(L, param1, param2, periodic):
    return sparseEgs(L, param1, param2, periodic)

ferro_tasks = []
para_tasks = []
Ls = list(range(2, 18))

for L in Ls:
    # Ferromagnetic calculations
    a = delayed_sparseEgs(L, 1, 0.3, False)
    b = delayed_sparseEgs(L+2, 1, 0.3, False)
    ferro_tasks.append((b - a) / 2)
    
    # Paramagnetic calculations
    a = delayed_sparseEgs(L, 1, 1.7, False)
    b = delayed_sparseEgs(L+2, 1, 1.7, False)
    para_tasks.append((b - a) / 2)
    
    
with ProgressBar():
    ferro_mean, para_mean = compute(ferro_tasks, para_tasks, scheduler='threads')    

In [ ]:
plt.plot(Ls, ferroOPEN, label=r'open, $\epsilon_{gs}(L)/L$', linestyle="dashdot", color='xkcd:royal blue')
plt.plot(Ls, ferroPERIODIC, label=r"periodic, $\epsilon_{gs}(L)/L$", color="xkcd:bright red")
plt.plot(Ls, ferro_mean, label=r'open mean $\frac{\epsilon_{gs}(L+2)-\epsilon_{gs}(L)}{2}$', color='xkcd:green', linestyle="dashdot")


plt.legend()
plt.xlabel('L')
plt.ylabel(r'Bulk energy per site')
plt.title( r'Bulk energy dependence on $L$ for ferro-magnetic phase ($h=0.3$)')
plt.savefig(os.path.join(directory, 'ferro_open_periodic_mean.png'), dpi=400)
plt.show()



plt.plot(Ls, paraOPEN, label=r'open, $\epsilon_{gs}(L)/L$', linestyle="dashdot", color='xkcd:royal blue')
plt.plot(Ls, paraPERIODIC, label=r"periodic, $\epsilon_{gs}(L)/L$", color="xkcd:bright red")
plt.plot(Ls, para_mean, label=r'open mean $\frac{\epsilon_{gs}(L+2)-\epsilon_{gs}(L)}{2}$', color='xkcd:green', linestyle="dashdot")

plt.legend()
plt.xlabel('L')
plt.ylabel(r'Bulk energy per site')
plt.title( r'Bulk energy dependence on $L$ for ferro-magnetic phase ($h=0.3$)')
plt.savefig(os.path.join(directory, 'para_open_periodic_mean.png'), dpi=400)
plt.show()

# 4.4 Finding the quantum phase transition

## Excitation gap curve

I will be using the dask package to speed up the computations so that it doesn't take unreasonably long time.

In [ ]:
client = Client(n_workers=20)
ProgressBar().register()

In [ ]:
@delayed

# function that can be inputted into dash
def diagonalize_h(h):
    H = sparseH(L, J, h, periodic)
    eigs = scipy.sparse.linalg.eigsh(H, k=num_states, which='SA', return_eigenvectors=False)
    return h, np.sort(eigs)

L = 20 # could have selected higher value, but it takes really long
J = 1.0
h_values = np.linspace(0, 2, 50)
periodic = True
num_states = 10 # more states can be selected
energies = np.zeros((len(h_values), num_states))

# Create a list of delayed tasks
tasks = [diagonalize_h(h) for h in h_values]

# Compute tasks with progress bar
results = compute(*tasks, scheduler='processes')

# Populate the energies array
for h, eigs in results:  # Iterate directly over results
    index = np.where(h_values == h)[0][0]
    energies[index, :] = eigs



### Plot of Excited state energies vs field strength

Plotting the first 9 excited staes

In [ ]:
excitation_energies = energies[:, 1:] - energies[:, 0, np.newaxis]
    
plt.figure(figsize=(10, 8))
plt.rcdefaults()
plt.rcParams['figure.dpi'] = 400

# Define line styles and markers
line_styles = ['-', '--', '-.', ':']
markers = ['o', '^', 's', '*', 'p', 'D', 'x', '+']

for i in range(1, num_states):
    plt.plot(h_values, excitation_energies[:, i-1], 
             label=f'Excited State {i}',
             linestyle=line_styles[i % len(line_styles)],  # Cycle through line styles
             marker=markers[i % len(markers)],  # Cycle through markers
             linewidth=1.5,
             markersize=4,
             alpha=0.8)
plt.xlabel('$h$')
plt.ylabel(r'$\varepsilon_{\text{excited}}$-$\varepsilon_{\text{gs}}$')
plt.title('Excitation Energies relative to ground-state vs. Magnetic Field Strength')
plt.savefig(os.path.join(directory, 'excited_states2.png'), dpi=400)
plt.show()


## Isolating the first excited state

Power law

In [ ]:
excitation_gap = energies[:, 1] - energies[:, 0]

line_styles = ['-', '--', '-.', ':']
markers = ['o', '^', 's', '*', 'p', 'D', 'x', '+']


X=(h_values-1)[25:]
Y=excitation_gap[25:]

def power_law(x, a, b, c):
    return a * np.power(x, b) + c 

params, covariance = scipy.optimize.curve_fit(power_law, X, Y)

a, b, c = params
print('a: ', a)
print('b: ', b)
print('c ', c)

In [ ]:
h_contvalues=np.linspace(1, 2, 1000)
Xcont=h_contvalues-1
Ycont=a*Xcont**b+c

plt.plot(Xcont, Ycont, label='fit', color='xkcd:bright red')

plt.plot((h_values-1)[25:], excitation_gap[25:], 
             label=f'Excited State {1}',
             linestyle=line_styles[1 % len(line_styles)],  # Cycle through line styles
             marker=markers[1 % len(markers)],  # Cycle through markers
             linewidth=1.5,
             markersize=4,
             alpha=0.8, color='xkcd:royal blue')

        
            
    
plt.xlabel(r'$h-h_c$')
plt.ylabel(r'Excitation gap, $\Delta$')
plt.legend()
plt.title('Excitation gap of the first excited state vs. Magnetic Field Strength')
plt.savefig(os.path.join(directory, 'excitationgap.png'), dpi=400)
plt.show()


In [ ]:
h_values = np.linspace(0, 2, 50)

# Define line styles and markers
line_styles = ['-', '--', '-.', ':']
markers = ['o', '^', 's', '*', 'p', 'D', 'x', '+']


plt.loglog(np.abs(h_values-1)[0:26], excitation_energies[:, 2-1][0:26], 
             label=f'Excited State {2}',
             linestyle=line_styles[1 % len(line_styles)],  # Cycle through line styles
             marker=markers[1 % len(markers)],  # Cycle through markers
             linewidth=1.5,
             markersize=4,
             alpha=0.8)
a=np.log(excitation_energies[:, 2-1][0]/excitation_energies[:, 2-1][10])
b=np.log(np.abs(h_values-1)[0]/np.abs(h_values-1)[10])



print('slope = ', a/b)



plt.loglog(np.abs(h_values-1)[26:], excitation_energies[:, 1-1][26:], 
             label=f'Excited State {1}',
             linestyle=line_styles[1 % len(line_styles)],  # Cycle through line styles
             marker=markers[1 % len(markers)],  # Cycle through markers
             linewidth=1.5,
             markersize=4,
             alpha=0.8)

a=np.log(excitation_energies[:, 1-1][49]/excitation_energies[:, 1-1][49-10])
b=np.log(np.abs(h_values-1)[49]/np.abs(h_values-1)[49-10])



print('slope = ', a/b)

            
    
plt.xlabel(r'$|h-h_c|$')
plt.ylabel(r'Excitation gap $\Delta$')
plt.title('Excitation Energies vs. Magnetic Field Strength')
plt.legend()
plt.savefig(os.path.join(directory, 'loglogexcited_states1,2.png'), dpi=400)
plt.show()


## Fidelity Curve

In [ ]:
L = 8
J = 1.0
h_values = np.linspace(0, 2, 100)
periodic = True
num_states = 1
energies = np.zeros((len(h_values), num_states))


def diagonalize_h(h, num_states_to_track=6):
    H = sparseH(L, J, h, periodic)
    # Compute more than one state to track the ground state
    eigs, vecs = scipy.sparse.linalg.eigsh(H, k=num_states_to_track, which='SA')
    return eigs, vecs

# The eigensolver kept getting confused between the degenerate ground_states, So I just made it track the state during the iterations
initial_h = h_values[0]
initial_eigs, initial_vecs = diagonalize_h(initial_h, num_states_to_track=2)
vec_prev = initial_vecs[:, 0]  # The initial ground state vector

tasks = [delayed(diagonalize_h)(h) for h in h_values[1:]]

results = compute(*tasks, scheduler='processes')

fidelities = np.zeros(len(h_values) - 1)

# Calculate fidelities using state tracking
for i, (eigs, vecs) in enumerate(results):
    overlaps = np.abs(np.dot(vecs.T.conj(), vec_prev))  
    max_overlap_index = np.argmax(overlaps) 
    vec_current = vecs[:, max_overlap_index] 
    fidelity = overlaps[max_overlap_index] 
    fidelities[i] = fidelity
    vec_prev = vec_current 

I ended up still having to truncate the value of h because the solver was still getting confused for small h, need to figure out why. But the phase transition is clear though.

In [ ]:
plt.rcParams['figure.dpi'] = 400
plt.plot(h_values[:-1], fidelities, '-o', color='xkcd:bright red')
plt.xlabel('$h$')
plt.ylabel('Fidelity $|\\langle \\psi_{gs}(h)|\\psi_{gs}(h + \\delta h)\\rangle|$')
plt.title('Fidelity as a Function of $h$')
plt.savefig(os.path.join(directory, 'fidelity.png'), dpi=400)
plt.show()

## 4.5 Study of magnetic ordering

The Correlation function $C_{zz}(r)$

In [ ]:
def Czz(vec, r):
    """
    generates the correlation function Czz(r) between sites 1 and 1+r
    
        Parameters:
            vec (np.ndarray): eigenvector
            r (integer): site
            
        Returns:
            C (float): correlation of sigma_z between site 1 and 1+r
    """
    
    dim = len(vec) 
    Cvec = np.zeros((dim, 1)) #initialize
    
    for alpha in range(dim): # iterate over basis
        
        if (alpha & (1 << (r))) == (2**(r))*(alpha & (1 << 0)):
            
            Cvec[alpha] = vec[alpha] # if site 1 and 1+r have spin, keep the coefficient unchanged
            
        else:
            Cvec[alpha] = -vec[alpha] # else multiply it by -1
    
    C=np.dot(np.conj(vec).T, Cvec)[0][0] # dot product of vector with the vector output from the operator action
    
    return C

Plot of $C^{zz}(r)$ as a function of r for $L=5,10,15,20$

In [ ]:
L_values = [5, 10, 15, 20]
hs = [0.3, 1, 1.7]
line_styles = ['-', '--', '-.', ':']
markers = ['o', '^', 's', '*', 'p', 'D', 'x', '+']

fig, axs = plt.subplots(2, 2, figsize=(12, 10), constrained_layout=True)
axs = axs.flatten() 
labels_handles = {}

for idx, L in enumerate(L_values):
    ax = axs[idx]

    for h in hs:
        H = sparseH(L, 1, h, True)
        _, vec = scipy.sparse.linalg.eigsh(H, k=1, which='SA')
        R = list(range(1, L))
        Cvecs = []

        for r in R:
            Cvecs.append(Czz(vec, r))

        if h == 0.3:
            linestyle = line_styles[1]
            marker = markers[1]
            label = "ferromagnetic phase"
        elif h == 1:
            linestyle = line_styles[2]
            marker = markers[2]
            label = "critical point"
        elif h == 1.7:
            linestyle = line_styles[3]
            marker = markers[4]
            label = "paramagnetic phase"

        line, = ax.plot(R, Cvecs, linestyle=linestyle, marker=marker, linewidth=1.5, markersize=4, alpha=0.8)
        labels_handles[label] = line

        ax.set_xlabel('r')
        ax.set_ylabel(r'Correlation $C^{zz}(r)=\langle \sigma_1^z \sigma_{1+r}^z \rangle$')
        ax.set_title(r'$C^{zz}(r)$ vs r for' +f' $L={L}$')

fig.legend(labels_handles.values(), labels_handles.keys(), loc='lower center', ncol=3, bbox_to_anchor=(0.5, 0))
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(os.path.join(directory, 'Czz_subplot.png'), dpi=400)
plt.show()


Plot of $\langle (M/L)^2 \rangle$ as a function of L

In [ ]:
Ls=list(range(2,21))
hs=[0.3, 1, 1.7]
line_styles = ['-', '--', '-.', ':']
markers = ['o', '^', 's', '*', 'p', 'D', 'x', '+']
for h in hs:
    
    M=[]
    
    for L in tqdm(Ls, desc=f'Progress for h={h}'):
    
        H = sparseH(L, 1, h, True)
        _, vec = scipy.sparse.linalg.eigsh(H, k=1, which='SA')
        
        R=list(range(0,L))
        Cs=[]
    
        for r in R:
            Cs.append(Czz(vec, r))
        
        m=(1/L)*np.sum(Cs)
        M.append(m)
        
        if h == 0.3:
            linestyle = line_styles[1]
            marker = markers[1]
            label = "ferromagnetic phase"
        elif h == 1:
            linestyle = line_styles[2]
            marker = markers[2]
            label = "critical point"
        elif h == 1.7:
            linestyle = line_styles[3]
            marker = markers[4]
            label = "paramagnetic phase"
        
    plt.plot(Ls, M, label=label, linestyle=linestyle, marker=marker, linewidth=1.5,
             markersize=4,
             alpha=0.8)
    
    
        
plt.xlabel('L')
plt.ylabel(r'$\langle (M/L)^2 \rangle$')
plt.title(r'$\langle (M/L)^2 \rangle$ vs L')
plt.legend()
plt.savefig(os.path.join(directory, f'ML2.png'), dpi=400)
plt.show()

Plot of $\sqrt{\sigma_1\sigma_{L/2}}$ as a function of L

In [ ]:
Ls=list(range(2,21))
hs=[0.3, 1, 1.7]
line_styles = ['-', '--', '-.', ':']
markers = ['o', '^', 's', '*', 'p', 'D', 'x', '+']
for h in hs:
    
    C=[]
    
    for L in tqdm(Ls, desc=f'Progress for h={h}'):
    
        H = sparseH(L, 1, h, True)
        _, vec = scipy.sparse.linalg.eigsh(H, k=1, which='SA')
        
        # Handling odd and even cases
        
        if L%2==0:
            R=int(L/2)
        else:
            R=1+(L//2)
            
        c=Czz(vec, R)
        C.append(c)
        
        if h == 0.3:
            linestyle = line_styles[1]
            marker = markers[1]
            label = "ferromagnetic phase"
        elif h == 1:
            linestyle = line_styles[2]
            marker = markers[2]
            label = "critical point"
        elif h == 1.7:
            linestyle = line_styles[3]
            marker = markers[4]
            label = "paramagnetic phase"
        
    plt.plot(Ls, C, label=label, linestyle=linestyle, marker=marker, linewidth=1.5,
             markersize=4,
             alpha=0.8)
    
    
        
plt.xlabel('L')
plt.ylabel(r'$\sigma_1\sigma_{L/2}$')
plt.title(r'$\sigma_1\sigma_{L/2}$ vs L')
plt.legend()
plt.savefig(os.path.join(directory, f'halfcorrelation.png'), dpi=400)
plt.show()




## 4.6 Study of Critical Correlations

The Correlation function $C^{xx}(r)$

In [ ]:
def Cxx(vec, r):
    
    """
    generates the correlation function Cxx(r) between sites 1 and 1+r
    
        Parameters:
            vec (np.ndarray): eigenvector
            r (integer): site
            
        Returns:
            C (float): correlation of sigma_x between site 1 and 1+r
    """
    
    dim = len(vec)
    Cvec = np.zeros((dim, 1)) # initialize
    
    for beta in range(dim): # iterate over basis
        
        alpha = (beta ^ (1<<0)) ^ (1<<r) # flip 1 and 1+r site to map the state to that which is connected by two bit flips
        
        Cvec[alpha] += vec[beta] # record this state
        
    C = np.dot(np.conj(vec).T, Cvec)
    
    return C[0][0]

The connected correlator $C^{xx}_{\text{conn}}(r)$

In [ ]:
def ConnCxx(vec, r):
    
    """
    generates the connected correlator Cxx(r) between sites 1 and 1+r <sigma_1 sigma_1+r> - <sigma_1><sigma_1+r>
    
        Parameters:
            vec (np.ndarray): eigenvector
            r (integer): site
            
        Returns:
            Cconn (float): connected correlator of sigma_x between site 1 and 1+r
    """
    
    # initialize
    dim = len(vec)
    Cvec1 = np.zeros((dim, 1))
    Cvec2 = np.zeros((dim, 1))
    
    for beta in range(dim): # iterate over basis
        
        alpha = beta ^ (1<<r) # flip 1+r site spin and record state
        Cvec1[alpha] += vec[beta]
        
        alpha = beta ^ (1<<0) # flip site 1 spin and record state
        Cvec2[alpha] += vec[beta]
    
    sigma1r = np.dot(np.conj(vec).T, Cvec1)[0][0]
    sigma1 = np.dot(np.conj(vec).T, Cvec2)[0][0]
    
    C = Cxx(vec, r)
    Cconn = C-sigma1*sigma1r # calculate the connected correlator

    return Cconn

In [ ]:
L = 20
hs=[0.3, 1, 1.7]
line_styles = ['-', '--', '-.', ':']
markers = ['o', '^', 's', '*', 'p', 'D', 'x', '+']

for h in hs:
    
    
    H = sparseH(L, 1, h, True)
    _, vec = scipy.sparse.linalg.eigsh(H, k=1, which='SA')
    R=list(range(1,L))
    Cvecs=[]
    
    for r in tqdm(R, desc=f'Progress for h={h}'):
        Cvecs.append(Cxx(vec, r)[0][0])
        
    if h==0.3:
        plt.plot(R, Cvecs, label = "ferromagnetic phase", linestyle=line_styles[1 % len(line_styles)],
             marker=markers[1 % len(markers)],
             linewidth=1.5,
             markersize=4,
             alpha=0.8)
    if h==1:
        plt.plot(R, Cvecs, label = "critical point", linestyle=line_styles[2 % len(line_styles)],
             marker=markers[2 % len(markers)],
             linewidth=1.5,
             markersize=4,
             alpha=0.8)
    if h==1.7:
        plt.plot(R, Cvecs, label = "paramagnetic phase", linestyle=line_styles[5 % len(line_styles)],
             marker=markers[4 % len(markers)],
             linewidth=1.5,
             markersize=4,
             alpha=0.8)
        
        
    
        
plt.xlabel('r')
plt.ylabel(r'Correlation $C^{xx}(r)$')
plt.title(r'$C^{xx}(r)$ vs r for'+ f' L={L}')
plt.legend()
plt.savefig(os.path.join(directory, f'Cxx{L}.png'), dpi=400)
plt.show()

In [ ]:
L = 20
hs=[0.3, 1, 1.7]
line_styles = ['-', '--', '-.', ':']
markers = ['o', '^', 's', '*', 'p', 'D', 'x', '+']

for h in hs:
    
    H = sparseH(L, 1, h, True)
    _, vec = scipy.sparse.linalg.eigsh(H, k=1, which='SA')
    R=list(range(1,L))
    Cvecs=[]
    
    for r in tqdm(R, desc=f'Progress for h={h}'):
        Cvecs.append(ConnCxx(vec, r))
        
    if h == 0.3:
        linestyle = line_styles[1]
        marker = markers[1]
        label = "ferromagnetic phase"
    elif h == 1:
        linestyle = line_styles[2]
        marker = markers[2]
        label = "critical point"
    elif h == 1.7:
        linestyle = line_styles[3]
        marker = markers[4]
        label = "paramagnetic phase"
        
    plt.plot(R, Cvecs, label=label, linestyle=linestyle, marker=marker, linewidth=1.5,
             markersize=4,
             alpha=0.8)
    
    
plt.xlabel('r')
plt.ylabel(r'Correlation $C_\text{conn}^{xx}(r)$')
plt.title(r'$C_\text{conn}^{xx}(r)$ vs r for'+ f' L={L}')
plt.legend()
plt.savefig(os.path.join(directory, f'Connxx{L}.png'), dpi=400)
plt.show()

### Critical point correlation power law

## $C^{xx}_\text{conn}$

In [ ]:
L = 20
h = 1
H = sparseH(L, 1, h, True)
_, vec = scipy.sparse.linalg.eigsh(H, k=1, which='SA')
R=list(range(1,L))
R=np.array(R)
Cvecs=[]

for r in tqdm(R, desc=f'Progress for h={h}'):
        Cvecs.append(ConnCxx(vec, r))
        
finiteR=(L/np.pi)*np.sin(((np.pi)*R)/L)

In [ ]:
def power_law(x, a, b):
    return a * np.power(x, b)

params, covariance = scipy.optimize.curve_fit(power_law, finiteR, Cvecs)

a, b = params
print('a: ', a)
print('b: ', b)

In [ ]:
Rcont = np.linspace(1, L-1, 1000)

finiteRcont = (L/np.pi)*np.sin(((np.pi)*Rcont)/L)

Cveccont = a * (finiteRcont)**b

plt.plot(finiteRcont, Cveccont, linestyle  ='-', linewidth=1.5, label='fit', color='xkcd:bright blue')

plt.plot(finiteR, Cvecs, linestyle=line_styles[3], marker=markers[4], linewidth=1.5,
             markersize=4,
             alpha=0.8, label=r'$C_\text{conn}^{xx}(r)$', color='xkcd:orange')
    
plt.xlabel(r'$\frac{L}{\pi}\ \sin\left(\frac{\pi r}{L} \right)$')
plt.ylabel(r'Correlation $C_\text{conn}^{xx}(r)$')
plt.title(r'Decay of $C_\text{conn}^{xx}(r)$  for'+ f' L={L} at the critical point')
plt.legend()
plt.savefig(os.path.join(directory, f'Connxx{L}powerlaw.png'), dpi=400)
plt.show()

## $C^{zz}_\text{conn}$

In [ ]:
def ConnCzz(vec, r):
    
    """
    generates the connected correlator Czz(r) between sites 1 and 1+r <sigma_1 sigma_1+r> - <sigma_1><sigma_1+r>
    
        Parameters:
            vec (np.ndarray): eigenvector
            r (integer): site
            
        Returns:
            Cconn (float): connected correlator of sigma_x between site 1 and 1+r
    """
    
    # initialize
    dim = len(vec)
    Cvec1 = np.zeros((dim, 1))
    Cvec2 = np.zeros((dim, 1))
    
    for alpha in range(dim): # iterate over basis
        
        if alpha & (1 << 0) == 0: # if site 1 has spin up
            
            Cvec1[alpha] = vec[alpha] # keep unchanged
        
        else:
            
            Cvec1[alpha] = -vec[alpha] # else eigenvalue is -1, so multiply by -1
            
            
        if alpha & (1 << r) == 0: # if site 1+r has spin up
            
            Cvec2[alpha] = vec[alpha] # keep unchanged
        
        else: 
            
            Cvec2[alpha] = -vec[alpha] # else eigenvalue is -1, so multiply by -1

    
    sigma1 = np.dot(np.conj(vec).T, Cvec1)[0][0]
    sigma1r = np.dot(np.conj(vec).T, Cvec2)[0][0]
    
    C = Czz(vec, r)
    
    Cconn = C-sigma1*sigma1r # calculate the connected correlator
    
    return Cconn

In [ ]:
L = 20
h = 1
H = sparseH(L, 1, h, True)
_, vec = scipy.sparse.linalg.eigsh(H, k=1, which='SA')
R=list(range(1,L))
R=np.array(R)
Cvecs=[]

for r in tqdm(R, desc=f'Progress for h={h}'):
        Cvecs.append(ConnCzz(vec, r))
        
finiteR=(L/np.pi)*np.sin(((np.pi)*R)/L)

In [ ]:
def power_law(x, a, b):
    return a * np.power(x, b)

params, covariance = scipy.optimize.curve_fit(power_law, finiteR, Cvecs)

a, b = params
print('a: ', a)
print('b: ', b)

In [ ]:
Rcont = np.linspace(1, L-1, 1000)

finiteRcont = (L/np.pi)*np.sin(((np.pi)*Rcont)/L)

Cveccont = a * (finiteRcont)**b

plt.plot(finiteRcont, Cveccont, linestyle  ='-', linewidth=1.5, label='fit', color='xkcd:bright blue')

plt.plot(finiteR, Cvecs, linestyle=line_styles[3], marker=markers[4], linewidth=1.5,
             markersize=4,
             alpha=0.8, label=r'$C_\text{conn}^{xx}(r)$', color='xkcd:orange')
    
plt.xlabel(r'$\frac{L}{\pi}\ \sin\left(\frac{\pi r}{L} \right)$')
plt.ylabel(r'Correlation $C_\text{conn}^{zz}(r)$')
plt.title(r'Decay of $C_\text{conn}^{zz}(r)$  for'+ f' L={L} at the critical point')
plt.legend()
plt.savefig(os.path.join(directory, f'Connzz{L}powerlaw.png'), dpi=400)
plt.show()

## 4.7 Making use of Ising symmetry

In [ ]:
def parity(x):
    count = 0
    while x:
        x = x & (x - 1)  
        count += 1
    
    return count


In [ ]:
def symmetryH(L, J, h, periodic):

    dim=2 ** (L-1) # dimensions of the Hilbert space

    
    Heven_data = []
    Heven_rows = []
    Heven_cols = []

    Hodd_data = []
    Hodd_rows = []
    Hodd_cols = []
    
    for beta in range(dim): 
        
        num = parity(beta)
        
        Aeven = -h*((L-1)-2*num + (-1)**(num))
        Aodd = -h*((L-1)-2*num + (-1)**(num + 1))
        

        Heven_data.append(Aeven)
        Heven_rows.append(beta)
        Heven_cols.append(beta)
        
        Hodd_data.append(Aodd)
        Hodd_rows.append(beta)
        Hodd_cols.append(beta)
        
        for j in range(1, L-1):
            
            alpha = (beta ^ (1<<j-1)) ^ (1<<j)

            Heven_data.append(-J)
            Heven_rows.append(alpha)
            Heven_cols.append(beta)
        
            Hodd_data.append(-J)
            Hodd_rows.append(alpha)
            Hodd_cols.append(beta)
    
        
            
        alpha = beta ^ (1 << L-2)


        Heven_data.append(-J)
        Heven_rows.append(alpha)
        Heven_cols.append(beta)

        Hodd_data.append(-J)
        Hodd_rows.append(alpha)
        Hodd_cols.append(beta)

        if periodic and L > 1:

            alpha = beta ^ (1 << 0)

            Heven_data.append(-J)
            Heven_rows.append(alpha)
            Heven_cols.append(beta)

            Hodd_data.append(-J)
            Hodd_rows.append(alpha)
            Hodd_cols.append(beta)

    Heven_data = np.array(Heven_data, dtype=float) # convert the list into a np array
    Hodd_data = np.array(Hodd_data, dtype=float)
    
    Heven = scipy.sparse.csr_matrix((Heven_data, (Heven_rows, Heven_cols)), shape=(dim, dim), dtype=np.float64) # make it into a csr sparse matrix  
    Hodd = scipy.sparse.csr_matrix((Hodd_data, (Hodd_rows, Hodd_cols)), shape=(dim, dim), dtype=np.float64) # make it into a csr sparse matrix  
    
    return Heven, Hodd

Check if the values match

In [ ]:
L=10
J=1
h=4

Heven=symmetryH(L, J, h, True)[0]
Hodd=symmetryH(L, J, h, True)[1]
Hsparse=sparseH(L, J, h, True)

print(f'L={L}, h={h}, J={J}, periodic=True')

print('-------')

print('eigenvalues of H-, odd sector: ')

print(scipy.sparse.linalg.eigsh(Hodd, k=1, which='SA', return_eigenvectors=False))
print('-------')

print('eigenvalues of H+, even sector: ')

print(scipy.sparse.linalg.eigsh(Heven, k=1, which='SA', return_eigenvectors=False))

print('-------')

print('eigenvalues of H from previous calculations: ')

print(scipy.sparse.linalg.eigsh(Hsparse, k=1, which='SA', return_eigenvectors=False))

Nice, we have implemented correctly

### Graph for L=21

In [ ]:
L_values = [21] 
h_values = np.linspace(0, 2, 20) # the problem is symmetric about h = 0, so we can cut computation-time in half by computing only positive hground_state_energies_even = {L: [] for L in L_values}
ground_state_energies_even = {L: [] for L in L_values}
ground_state_energies_odd = {L: [] for L in L_values}
xkcd_colors=['xkcd:indigo', 'xkcd:royal blue', 'xkcd:bright green', 'xkcd:red']


for L in L_values:
    for h in tqdm(h_values, desc=f'Computing for L={L}'):
        H_even, H_odd = symmetryH(L, 1, h, periodic=True)
        ground_state_even = scipy.sparse.linalg.eigsh(H_odd, k=1, which='SA', return_eigenvectors=False)[0]
        ground_state_energies_even[L].append(ground_state_even)
        ground_state_odd = scipy.sparse.linalg.eigsh(H_odd, k=1, which='SA', return_eigenvectors=False)[0]
        ground_state_energies_odd[L].append(ground_state_even)


ground_state_energies = {L: [] for L in L_values}
for L in L_values:
       for e_even, e_odd in zip(ground_state_energies_even[L], ground_state_energies_odd[L]):
               ground_state_energies[L].append(min(e_even, e_odd))


In [ ]:
# Dashed line -> Open
# Solid line -> Periodic
        
for i, (L, energies) in enumerate(ground_state_energies.items()):
    plt.plot(h_values, energies, label=f'L={L}', color='xkcd:red')


plt.xlabel('field strength')
plt.ylabel(r'$E_{gs}$')
plt.title('Ground state energy versus magnetic field \n for various values of L')
plt.legend()
plt.savefig(os.path.join(directory, 'plot-L=21.png'), dpi=400)
plt.show()

In [ ]:
L_values = [5, 10, 15, 20] 
h_values = 0.3
excitation_gaps = {L: [] for L in L_values}

line_styles = ['-', '--', '-.', ':']
markers = ['o', '^', 's', '*', 'p', 'D', 'x', '+']

for L in L_values:
    for h in tqdm(h_values, desc=f'Computing for L={L}'):
        H_even, H_odd = symmetryH(L, 1, h, periodic=True)
        
        e_vals_even = scipy.sparse.linalg.eigsh(H_even, k=2, which='SA', return_eigenvectors=False)
        e_vals_odd = scipy.sparse.linalg.eigsh(H_odd, k=2, which='SA', return_eigenvectors=False)
        
        all_e_vals = np.hstack([e_vals_even, e_vals_odd])
        two_smallest = np.partition(all_e_vals, 1)[:2]
        excitation_gap = two_smallest[1] - two_smallest[0]
        excitation_gaps[L].append(excitation_gap)

In [ ]:
for L in L_values:
    plt.plot(h_values, excitation_gaps[L], label=f'L={L}',
            linestyle=line_styles[L % len(line_styles)],
             marker=markers[L % len(markers)],
             linewidth=1.5,
             markersize=4,
             alpha=0.8)

plt.xlabel('Magnetic Field Strength, h')
plt.ylabel(r'Excitation Gap $\Delta =\varepsilon_1-\varepsilon_0$')
plt.title(r'Exponentially small splitting in the ferromagnetic phase')
plt.legend()
plt.savefig(os.path.join(directory, 'ferro-excitationgap.png'), dpi=400)
plt.show()

In [ ]:
L_values = list(range(10, 20))
h = 0.7
excitation_gaps = {L: [] for L in L_values}

for L in L_values:
    H_even, H_odd = symmetryH(L, 1, h, periodic=True)
    e_vals_even = scipy.sparse.linalg.eigsh(H_even, k=2, which='SA', return_eigenvectors=False)
    e_vals_odd = scipy.sparse.linalg.eigsh(H_odd, k=2, which='SA', return_eigenvectors=False)
    
    all_e_vals = np.hstack([e_vals_even, e_vals_odd])
    two_smallest = np.partition(all_e_vals, 1)[:2]
    excitation_gap = two_smallest[1] - two_smallest[0]
    excitation_gaps[L].append(excitation_gap)


In [ ]:
Y=[]
for L in L_values:
    Y.append(excitation_gaps[L][0])

def power_law(x, a, b):
    return a * (1/np.exp(b*x))

params, params_covariance = scipy.optimize.curve_fit(power_law, L_values, Y)
Lconts=np.linspace(10,20, 1000)
fitted_values = power_law(Lconts, *params)

plt.figure(figsize=(8, 6))
plt.scatter(L_values, Y, color='red', label='Data Points')
plt.plot(Lconts, fitted_values, label=f'Fitted curve: $\Delta = {params[0]:.2f} \cdot e^{{ -{params[1]:.2f} L}}$', color='blue')
plt.xlabel('System Size, L')
plt.ylabel(r'Excitation Gap $\Delta = \varepsilon_1 - \varepsilon_0$')
plt.title(f'Fit of a Power Law to the Excitation Gap at h={h}')
plt.savefig(os.path.join(directory, 'ferro-fitgap.png'), dpi=400)
plt.legend()
plt.show()
    

    

In [ ]:
L_values = list(range(10, 20))  # from 10 to 20 inclusive
h = 0.7
excitation_gaps = {L: [] for L in L_values}

for L in L_values:
    H_even, H_odd = symmetryH(L, 1, h, periodic=True)
    e_vals_even = scipy.sparse.linalg.eigsh(H_even, k=2, which='SA', return_eigenvectors=False)
    e_vals_odd = scipy.sparse.linalg.eigsh(H_odd, k=2, which='SA', return_eigenvectors=False)
    
    all_e_vals = np.hstack([e_vals_even, e_vals_odd])
    two_smallest = np.partition(all_e_vals, 1)[:2]
    excitation_gap = two_smallest[1] - two_smallest[0]
    excitation_gaps[L].append(excitation_gap)


In [ ]:
Y=[]
for L in L_values:
    Y.append(excitation_gaps[L][0])

def power_law(x, a, b):
    return a * (1/np.exp(b*x))

params, params_covariance = scipy.optimize.curve_fit(power_law, L_values, Y)
Lconts=np.linspace(10,20, 1000)
fitted_values = power_law(Lconts, *params)

plt.figure(figsize=(8, 6))
plt.scatter(L_values, Y, color='red', label='Data Points')
plt.plot(Lconts, fitted_values, label=f'Fitted curve: $\Delta = {params[0]:.2f} \cdot e^{{ -{params[1]:.2f} L}}$', color='blue')
plt.xlabel('System Size, L')
plt.ylabel(r'Excitation Gap $\Delta = \varepsilon_1 - \varepsilon_0$')
plt.title(f'Fit of a Power Law to the Excitation Gap at h={h}')
plt.legend()
plt.savefig(os.path.join(directory, 'ferro-fitgap.png'), dpi=400)
plt.show()
    

    